### Set up pipeline with basic imports, data directories, and file basenames.

In [ ]:
import os
from benbiohelpers.DataPipelineManagement.GenomeManager import getGenomeFastaFilePath
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import getExternalDataDirectory as getMutperiodExternalDataDirectory

hg19FastaFilePath = getGenomeFastaFilePath("hg19")
mutperiodHg19Directory = getMutperiodExternalDataDirectory()

smmSeqDataDirectory = os.path.join(getDataDir(), "Adar_smm-seq")
smmSeqBaseDataFilePath = os.path.join(smmSeqDataDirectory, "smm-seq_BPDE_all_reps.bed")

### Parse the mutation data for mutperiod input

In [ ]:
from mutperiodpy.input_parsing.ParseCustomBed import parseCustomBed
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import DataTypeStr
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory

if getFilesInDirectory(smmSeqDataDirectory, DataTypeStr.mutations + ".bed", searchRecursively = False) is None:
    print("Parsed data not found. Generating...")
    smmSeqMutperiodInputFilePaths = parseCustomBed([smmSeqBaseDataFilePath], hg19FastaFilePath)
else:
    smmSeqMutperiodInputFilePaths = [getFilesInDirectory(smmSeqDataDirectory, DataTypeStr.mutations + ".bed", searchRecursively = False)]
    print(f"Found parsed data at {smmSeqMutperiodInputFilePaths[0]}. Continuing.")

### Run the mutperiod pipeline to determine translational and rotational periodicity of the data.

In [ ]:
from mutperiodpy.RunAnalysisSuite import runAnalysisSuite

nucleosomeMapFilePaths = list()
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_hybrid_nuc_map", "hg19_hybrid_nuc_map.bed"))
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map", "hg19_LCL_MNase_nuc_map.bed"))
# nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_euchromatin",
#                                            "hg19_LCL_MNase_nuc_map_euchromatin.bed"))
# nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_heterochromatin",
#                                            "hg19_LCL_MNase_nuc_map_heterochromatin.bed"))
# nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_stringent_euchromatin",
#                                            "hg19_LCL_MNase_nuc_map_stringent_euchromatin.bed"))
# nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_NHF1_MNase_nuc_map", "hg19_NHF1_MNase_nuc_map.bed"))

nucleosomeMapNames = [os.path.basename(nucleosomeMapFilePath).rsplit('.', 1)[0] for nucleosomeMapFilePath in nucleosomeMapFilePaths]

runAnalysisSuite(smmSeqMutperiodInputFilePaths, nucleosomeMapNames, normalizationMethod = "Trinuc/Quadrunuc", customBackgroundDir = None,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True)

In [ ]:
from mutperiodpy.RunNucleosomeMutationAnalysis import runNucleosomeMutationAnalysis

normalizedNucleosomeCountsFilePaths = list()
for nucleosomeMapName in nucleosomeMapNames:
    normalizedNucleosomeCountsFilePaths += getFilesInDirectory(os.path.join(smmSeqDataDirectory, nucleosomeMapName), DataTypeStr.normNucCounts + ".tsv")

runNucleosomeMutationAnalysis(normalizedNucleosomeCountsFilePaths,
                              outputFilePath = os.path.join(smmSeqDataDirectory, "smm-seq_BPDE_trinuc_normalized_periodicity_data.tsv"),
                              overridePeakPeriodicityWithExpected = False, alignStrands = False)